<a href="https://colab.research.google.com/github/mmaronbr/alura-imersao-ia/blob/main/organizador_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependências da Aplicação

Dependências da Generative AI

In [ ]:
!pip install -U -q google.generativeai

Dependências do Gmail

In [ ]:
!pip install -U -q google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install -U -q pyngrok

# Importação das Bibliotecas

Bibliotecas de uso comum

In [ ]:
from google.colab import userdata

Bibliotecas utilizadas para acesso à API da Generative AI

In [ ]:
import google.generativeai as genai

Bibliotecas utilizadas para acesso ao Gmail

In [ ]:
import os.path
import pickle
import requests

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

from pyngrok import ngrok

# Configurações

## Generative AI

Definição da Chave da API da Generative AI

**IMPORTANTE!** Cadastre a secret `GOOGLE_API_KEY` na Seção de Secrets do Colab.

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Gmail

Configurações para acesso ao Gmail

**IMPORTANTE!** Cadastre as secrets: `CLIENT_ID`, `CLIENT_SECRET`,  `PROJECT_ID` e `NGROK_AUTHTOKEN` na Seção de Secrets do Colab.

Para obter o `CLIENT_ID`, `CLIENT_SECRET` e `PROJECT_ID` siga as orientações em: [Google Developers - Guia de Criação de Cliente OAuth](https://developers.google.com/workspace/guides/create-credentials?hl=pt-br#oauth-client-id)

Para o `NGROK_AUTHTOKEN`, registre-se em: [https://ngrok.com/](https://ngrok.com/)



Definição das constantes de autenticação/autorização

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

CLIENT_ID = userdata.get('CLIENT_ID')
CLIENT_SECRET = userdata.get('CLIENT_SECRET')
PROJECT_ID = userdata.get('PROJECT_ID')
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')

# Arquivo que armazena as configurações de acesso via OAuth2.0
CREDENTIALS_FILE = 'credentials.json'
# Arquivo que armazenará o token de acesso aos serviços da Google
TOKEN_PICKLE_FILE = 'token.pickle'
# Arquivo que armazena as configurações ngrok
NGROK_FILE = 'ngrok.yml'

# Configurações do Servidor Flow
PORT = 9999

Criação do arquivo de configuração do ngrok

In [ ]:
NGROK_TMP =f'''authtoken: {NGROK_AUTHTOKEN}
version: 2

tunnels:
  oauth_tunnel:
    proto: http
    addr: {PORT}
'''
# Cria o arquivo de Configurção do ngrok
with open(NGROK_FILE, "w") as n_file:
  n_file.write(NGROK_TMP)

Inicia o serviço ngrok em background

Para consultar os endpoints ativos, consulte: [Dashboard do Ngrok](https://dashboard.ngrok.com/cloud-edge/endpoints)

In [ ]:
#!nohup ngrok --config ngrok.yml start oauth_tunnel > nohup.out 2>&1 &
!nohup ngrok --config ngrok.yml > nohup.out 2>&1 &

Obtém o endereço público para comunicação com o Google

In [ ]:
ngrok.set_auth_token(NGROK_AUTHTOKEN)

public_url = ngrok.connect(PORT)
print(f"URL Público: {public_url.public_url}")
print("\nATENÇÃO! Cadastre o novo domínio do ngrok como um domínio válido para sua aplicação.\n")
print("Acesse: https://console.cloud.google.com/apis/credentials/consent")

configured = "não"

while configured.lower() != "sim":
  configured = input("Adicionou o endereço no ngrok como domínio autorizado?  ")

In [ ]:
JSON_TMP = f'''{{
    "installed": {{
      "client_id": "{CLIENT_ID}",
      "project_id": "{PROJECT_ID}",
      "auth_uri": "https://accounts.google.com/o/oauth2/auth",
      "token_uri": "https://oauth2.googleapis.com/token",
      "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
      "client_secret": "{CLIENT_SECRET}",
      "redirect_uris": ["{public_url.public_url}"]
    }}
  }}'''


# Cria o arquivo de Credenciais do Google
with open(CREDENTIALS_FILE, "w") as cred_file:
  cred_file.write(JSON_TMP)


# Definição das Funções

In [ ]:
def authenticate():
    creds = None

    # Recupera o token existente, caso o arquivo exista
    if os.path.exists(TOKEN_PICKLE_FILE):
        with open(TOKEN_PICKLE_FILE, 'rb') as token:
            creds = pickle.load(token)

    # Se as credenciais são inválidas, permite ao usuário se autenticar
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            # O Flow permite a conexão via OAuth2.0 para aplicações Desktop
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDENTIALS_FILE, SCOPES)
            creds = flow.run_local_server(port=PORT, open_browser=False)  # Use headless mode)

        # Save the credentials for the next run
        with open(TOKEN_PICKLE_FILE, 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [ ]:
def get_inbox_emails():
    """Fetch emails from Gmail inbox."""
    creds = authenticate()
    service = build('gmail', 'v1', credentials=creds)

    # Call the Gmail API to fetch emails
    results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
    messages = results.get('messages', [])

    if not messages:
        print('Nenhuma mensagem encontrada! :)')
    else:
        print(f'''Você tem: {len(messages)} mensagens!''')
        print('Messages:')
        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            # Decode the message payload
            payload = msg['payload']
            headers = payload['headers']
            subject = [header['value'] for header in headers if header['name'] == 'Subject'][0]
            snippet = msg['snippet']
            print(f'Subject: {subject}, Snippet: {snippet}')

# Testes

Verifica se as configurações foram realizadas com sucesso para Generative AI

In [ ]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("Give me python code to sort a list")
print(response.text)

Verifica as configurações de autorização com o Google

In [ ]:
credentials=authenticate()

Verifica se a comunicação com a API do Gmail está funcional. O script abaixo irá imprimir todos os marcadores (labels) do Gmail do usuário.

In [ ]:
try:
    # Chama a API do Gmail
    service = build("gmail", "v1", credentials=authenticate())
    results = service.users().labels().list(userId="me").execute()
    labels = results.get("labels", [])

    if not labels:
      print("Rótulos não encontrados")
    print("Labels:")
    for label in labels:
      print(label["name"])

except HttpError as error:
  print(f"An error occurred: {error}")

# Organizador de E-mails

In [ ]:
get_inbox_emails()